In [ ]:
!pip install nltk
!pip install stanza

In [ ]:
import math
import nltk
from nltk import word_tokenize
from google.colab import drive
import pandas as pd
import numpy as np
import csv
import stanza
nltk.download('punkt')
stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,sentiment')
drive.mount('/content/drive')

In [ ]:
filenames = []
df = pd.read_csv('/content/drive/MyDrive/original_data/patient_data/patient_info.csv')
filenames = list(df['name'])

In [ ]:
feature_list=[]
old_data=[]
with open('/content/drive/MyDrive/original_data/patient_time_npl.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    feature_list=next(csv_reader)
    for row in csv_reader:
       old_data.append(row)
add_feature=['mean_ttr','total_ttr','max_ttr','mean_mattr','total_mattr','max_mattr','mean_brunet_index','total_brunet_index','max_brunet_index','mean_honore_statistic','total_honore_statistic','max_honore_statistic']
for i in range(0,len(add_feature)):
  feature_list.append(add_feature[i])

In [ ]:
add_feature=['good_emoji','no_emoji','bad_emoji']
for i in range(0,len(add_feature)):
  feature_list.append(add_feature[i])

In [ ]:
def emoji_analysis(line):
  sentence=line.split('.')
  good_emoji=0
  no_emoji=0
  bad_emoji=0
  for i in range(0,len(sentence)-1):
    try:
      doc = nlp(sentence[i])
      result=doc.sentences[0].sentiment
      if(result==0):
        bad_emoji+=1
      elif(result==1):
        no_emoji+=1
      else:
        good_emoji+=1
    except:
      continue
  return good_emoji,no_emoji,bad_emoji
#emoji_analysis("I hate you.")

In [ ]:
def voc_richness(text):
  sentences=text.split('.')
  ttr=[]
  mattr=[]
  brunet_index=[]
  honore_statistic=[]
  for i in range(0,len(sentences)):
    tokens = word_tokenize(text.lower())
    ttr.append(len(set(tokens)) / len(tokens))
    def calculate_mattr(tokens, window_size):
        ttrs = [len(set(tokens[i:i + window_size])) / window_size for i in range(len(tokens) - window_size + 1)]
        return sum(ttrs) / len(ttrs) if len(ttrs) > 0 else 0
    mattr.append(calculate_mattr(tokens, window_size=15))
    def calculate_brunet_index(total_types, total_tokens):
        return total_types / (total_tokens ** 0.165)
    def calculate_honore_statistic(total_types, total_tokens):
        return 100 * (math.log(total_tokens) / max(1, (1 - (total_types / total_tokens))))
    total_types = len(set(tokens))
    total_tokens = len(tokens)
    brunet_index.append(calculate_brunet_index(total_types, total_tokens))
    honore_statistic.append(calculate_honore_statistic(total_types, total_tokens))
  return ttr,mattr,brunet_index,honore_statistic

In [ ]:
for i in range(0,len(filenames)):
  try:
    print(filenames[i])
    source_file_path='/content/drive/MyDrive/original_data/ASR/'+filenames[i]+".txt"
    f1 = open(source_file_path, "r")
    line1 = f1.readline()
    f1.close()
    #ttr,mattr,brunet_index,honore_statistic=voc_richness(line1)
    good_emoji,no_emoji,bad_emoji=emoji_analysis(line1)
    for j in range(0,len(old_data)):
      if(old_data[j][0]==filenames[i]):
        print("complete",filenames[i])
        #old_data[j].append(np.mean(ttr))
        #old_data[j].append(np.sum(ttr))
        #old_data[j].append(np.max(ttr))
        #old_data[j].append(np.mean(mattr))
        #old_data[j].append(np.sum(mattr))
        #old_data[j].append(np.max(mattr))
        #old_data[j].append(np.mean(brunet_index))
        #old_data[j].append(np.sum(brunet_index))
        #old_data[j].append(np.max(brunet_index))
        #old_data[j].append(np.mean(honore_statistic))
        #old_data[j].append(np.sum(honore_statistic))
        #old_data[j].append(np.max(honore_statistic))
        old_data[j].append(good_emoji)
        old_data[j].append(no_emoji)
        old_data[j].append(bad_emoji)
  except:
    continue

In [ ]:
for i in range(0,len(old_data)):
  all_emoji=sum(old_data[i][feature_list.index('good_emoji'):feature_list.index('bad_emoji')+1])
  for j in range(feature_list.index('good_emoji'),feature_list.index('bad_emoji')+1):
    if(all_emoji!=0):
      old_data[i][j]/=all_emoji
    else:
      old_data[i][j]=0

In [ ]:
with open('/content/drive/MyDrive/original_data/patient_time_npl.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(feature_list)
  for i in range(0,len(old_data)):
    writer.writerow(old_data[i])